In [ ]:
!pip install langchain langchain-community langchain-text-splitters langchain-google-genai langchain-chroma google-generativeai chromadb

INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 23.5 MB/s e

In [ ]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 8.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata;

api_key = userdata.get('google.studio.ai');

In [ ]:
!ls -all

total 20
drwxr-xr-x 1 root root 4096 Mar 14 13:46 .
drwxr-xr-x 1 root root 4096 Mar 14 13:38 ..
drwxr-xr-x 4 root root 4096 Mar 12 13:34 .config
drwx------ 6 root root 4096 Mar 14 13:46 drive
drwxr-xr-x 1 root root 4096 Mar 12 13:35 sample_data


In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader, JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

os.environ["GOOGLE_API_KEY"] = api_key

loader = DirectoryLoader(
    path="/content/drive/MyDrive/Bundle",
    glob="*.txt",
    loader_cls=TextLoader
)

json_loader = JSONLoader(
    file_path="/content/drive/MyDrive/Bundle/experience.json",
    jq_schema='.[]',
    content_key="message"
)

documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
splits = text_splitter.split_documents(documents)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db"
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=os.environ["GOOGLE_API_KEY"],
    temperature=0.2
)

template = """
You are an AI assistant that helps answer recruiter questions based on the Dip's personal information.
Use ONLY the provided context to answer the question. If the information is not in the context, try to related information and infer the answer from the text.
Be concise, logical, respectful, and highlight the my strengths relevant to the question. Please use third person pronoun. Also whenever you are asked moral question then try to find something positive from my text and give it.

If possible can you please keep answer short and concise.

"experience.txt" has all information about Dip's experience so whenever skills or experience in something related question was asked refer to that file.

Context:
{context}

Recruiter's Question: {question}

Response:
"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def answer_recruiter_question(question):
    return rag_chain.invoke(question)

In [ ]:
question = "Does he have experience in LLMs";
response = answer_recruiter_question(question);
print(response);

He has experience with transformer models, which are the foundation of LLMs. He fine-tuned pre-trained transformer models for clickbait classification and generation, achieving 77.53% accuracy in classification and a 0.299 METEOR score in generation.  While this demonstrates his understanding of transformer architecture, it's not explicitly stated whether he has worked with Large Language Models specifically.
